In [1]:
import json
import pprint
import joblib
import pickle
import os
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
decoded_predictions = pd.DataFrame(columns = ["offer_id", "gender", "income", "reward", "channels", "difficulty", "duration", "offer_type"])
decoded_predictions

,offer_id,gender,income,reward,channels,difficulty,duration,offer_type


In [3]:
# Opening JSON file
f = open('offers_encoded.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  
# Iterating through the json
# list
for i in data:
    pprint.pprint(i)

{'channels': 1,
 'difficulty': 10,
 'duration': 7,
 'gender': 'gender',
 'income': 'income',
 'index': 0,
 'offer_id': 8,
 'offer_type': 2,
 'reward': 10}
{'channels': 0,
 'difficulty': 10,
 'duration': 5,
 'gender': 'gender',
 'income': 'income',
 'index': 1,
 'offer_id': 2,
 'offer_type': 2,
 'reward': 10}
{'channels': 0,
 'difficulty': 5,
 'duration': 5,
 'gender': 'gender',
 'income': 'income',
 'index': 2,
 'offer_id': 7,
 'offer_type': 2,
 'reward': 5}
{'channels': 3,
 'difficulty': 5,
 'duration': 7,
 'gender': 'gender',
 'income': 'income',
 'index': 3,
 'offer_id': 3,
 'offer_type': 2,
 'reward': 5}
{'channels': 1,
 'difficulty': 0,
 'duration': 3,
 'gender': 'gender',
 'income': 'income',
 'index': 4,
 'offer_id': 9,
 'offer_type': 1,
 'reward': 0}
{'channels': 3,
 'difficulty': 0,
 'duration': 4,
 'gender': 'gender',
 'income': 'income',
 'index': 5,
 'offer_id': 1,
 'offer_type': 1,
 'reward': 0}
{'channels': 0,
 'difficulty': 7,
 'duration': 7,
 'gender': 'gender',
 'incom

In [4]:
complete_knn = joblib.load('complete_offer.pkl')


In [5]:
def decode(dictionary):
    dictionary['offer_id'] = dictionary['offer_id'].map({8:'Offer 7: The Americano',
                             2:'Offer 2: The Cold Brew',
                            1:'Offer 8: The Espresso',
                            3:'Offer 6: The Pourover',
                            0:'Offer 9: The Macchiato',
                            5:'Offer 5: The French Press',
                            4:'Offer 4: The Mocha',
                            9:'Offer 3: The Latte',
                            7:'Offer 10: The Cappuccino',
                            6:'Offer 1: The Doppio'},
                             na_action=None)
    dictionary['gender'] = dictionary['gender'].map({0:'M',
                                                    1: 'O',
                                                    2: 'F'})
    dictionary['channels'] = dictionary['channels'].map({1: "email, mobile, social",
                                                         3: "web, email, mobile",
                                                         0: "web, email, mobile, social",
                                                         2: "web, email"})
    dictionary['offer_type'] = dictionary['offer_type'].map({1: "informational",
                                                            2: "bogo",
                                                            0: "discount"})
    dictionary['offer_completed_y_n'] = dictionary['offer_completed_y_n'].map({0:"Yes",
                                                                        1: "No"})
    global decoded_predictions
    decoded_predictions = decoded_predictions.append(dictionary)

In [6]:
g = int(2)
i = int(72000)

def predictions():
    for dictionary in data:
        dictionary.update(gender = g, income = i)
        complete_df = pd.DataFrame(dictionary, index=[0])[["offer_id", "gender", "income", "reward", "channels", "difficulty", "duration", "offer_type"]]
        complete_score = int(complete_knn.predict(complete_df))
        dictionary['offer_completed_y_n'] = complete_score
        new_dict = pd.DataFrame(dictionary, index=[0])
        decode(new_dict)

predictions()

In [7]:
decoded_predictions.sort_values('offer_completed_y_n', ascending = False).drop(columns='index')

,offer_id,gender,income,reward,channels,difficulty,duration,offer_type,offer_completed_y_n
0,Offer 7: The Americano,F,72000,10,"email, mobile, social",10,7,bogo,Yes
0,Offer 10: The Cappuccino,F,72000,5,"web, email, mobile, social",5,5,bogo,Yes
0,Offer 3: The Latte,F,72000,0,"email, mobile, social",0,3,informational,Yes
0,Offer 2: The Cold Brew,F,72000,10,"web, email, mobile, social",10,5,bogo,No
0,Offer 6: The Pourover,F,72000,5,"web, email, mobile",5,7,bogo,No
0,Offer 8: The Espresso,F,72000,0,"web, email, mobile",0,4,informational,No
0,Offer 5: The French Press,F,72000,3,"web, email, mobile, social",7,7,discount,No
0,Offer 4: The Mocha,F,72000,2,"web, email, mobile, social",10,10,discount,No
0,Offer 1: The Doppio,F,72000,2,"web, email, mobile",10,7,discount,No
0,Offer 9: The Macchiato,F,72000,5,"web, email",20,10,discount,No
